## Elo Rating method

This is the Elo method.  The example is taken from "Who's #1" by Langville and Meyer and the results match.  Note, the data includes the entire 2009 NFL season, including all playoff games.  

Reference: https://www.gautamnarula.com/rating/

In [22]:
#This cell contains the main calculations associated with the Elo method
#The first function calculates the probability of each team winning
#The second function takes those probabilities, and the outcome of the game to determine the new ratings

import math 
  
# Function to calculate the Probability 
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400)) 
  
# Function to calculate Elo rating 
# K is a constant. 
# Player A wins over Player B.  
# tie = true if tie, false otherwise
def EloRating(Ra, Rb, K, tie): 
    
    # To calculate the Winning 
    # Probability of Player B 
    Pb = Probability(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = Probability(Rb, Ra) 
  
    # Updating the Elo Ratings 
    if tie:
       Ra = Ra + K * (1/2 - Pa) 
        
       Rb = Rb + K * (1/2 - Pb) 
    else:        
       Ra = Ra + K * (1 - Pa) 
       Rb = Rb + K * (0 - Pb) 
    
    return Ra, Rb

In [23]:
pathGames = '/FILEPATH/' #filepath to game file
pathTeams = '/FILEPATH/' #filepath to team file
gameFilename = '.txt'
teamFilename = '.txt'

#We got our data from masseyratings.com, so reading the files is based on the structure of those files

### Load the team names into an array

In [24]:
import pandas as pd

teamNames = pd.read_csv(pathTeams + teamFilename, header = None)
numTeams = len(teamNames)

### Load the games

In [25]:
# columns of games are:
#	column 0 = days since 1/1/0000
#	column 1 = date in YYYYMMDD format
#	column 2 = team1 index
#	column 3 = team1 homefield (1 = home, -1 = away, 0 = neutral)
#	column 4 = team1 score
#	column 5 = team2 index
#	column 6 = team2 homefield (1 = home, -1 = away, 0 = neutral)
#	column 7 = team2 score

games = pd.read_csv(pathGames + gameFilename, header = None)
numGames = len(games)

### Create Elo ratings and calculate predictability

In [26]:
import numpy as np

K = 32

eloRatings = np.zeros(numTeams)

numberCorrectPredictions = 0

for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 # subtracting 1 since python indexes at 0
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 # subtracting 1 since python indexes at 0
    team2Score = games.loc[i, 7]

    # Check if prediction correct
    if team1Score > team2Score and eloRatings[team1ID] > eloRatings[team2ID]:
        numberCorrectPredictions += 1
    elif team2Score > team1Score and eloRatings[team2ID] > eloRatings[team1ID]:
        numberCorrectPredictions += 1
    elif team1Score == team2Score and eloRatings[team1ID] == eloRatings[team2ID]:
        numberCorrectPredictions += 1
    
    # Update ratings 
    if team1Score > team2Score:
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], K, False)
    elif team1Score < team2Score:
        team2Rating, team1Rating = EloRating(eloRatings[team2ID], eloRatings[team1ID], K, False)
    else:  
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], K, True)
        
    eloRatings[team1ID] = team1Rating
    eloRatings[team2ID] = team2Rating

### Sort and print the ranking of teams

In [27]:
#This cell prints the ratings and rankings
k = 0 
iSort = np.argsort(-eloRatings)

print('\n\n************** ELO Rating Method **************\n')
print('===========================')
print('Rank   Rating      Team   ')
print('===========================')
if k == 0:
    for i in range(numTeams):
        print(f'{i+1:4d}   {eloRatings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')
else:
    for i in range(k):
        print(f'{i+1:4d}   {eloRatings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')
    
print('')   # extra carriage return
    
print(f'Average Predictability: {numberCorrectPredictions/(numGames)*100:.2f}%')



************** ELO Rating Method **************

Rank   Rating      Team   
   1   230.89868   Oklahoma_City
   2   166.49674   Boston
   3   155.05892   Cleveland
   4   118.30882   LA_Clippers
   5   106.61023   Indiana
   6   92.74305   Minnesota
   7   86.72842   Golden_State
   8   85.23112   Milwaukee
   9   82.27257   LA_Lakers
  10   81.24448   Houston
  11   75.82264   Denver
  12   66.20988   New_York
  13   28.82729   Memphis
  14   27.63981   Detroit
  15   11.33565   Orlando
  16   6.27668   Chicago
  17   -9.29066   Portland
  18   -19.55258   Sacramento
  19   -31.81497   Atlanta
  20   -33.64353   Miami
  21   -46.15285   Dallas
  22   -56.05162   Phoenix
  23   -67.08389   San_Antonio
  24   -83.43978   Toronto
  25   -140.50753   Brooklyn
  26   -158.04351   New_Orleans
  27   -181.55195   Washington
  28   -184.91089   Philadelphia
  29   -204.04381   Utah
  30   -205.61743   Charlotte

Average Predictability: 62.33%
